In [ ]:
import torch;
import tiktoken;
import torch
import torch.nn as nn
import math


from MyTransformer import TestTransformer;

torch.manual_seed(123)



GPT_CONFIG = {
    "vocab_size": 50257,    #词汇表大小
    "max_seq_length": 256, #每一句训练数据的最大长度
    "embedding_dim": 768,   #嵌入向量的维度
    "n_heads": 12,          #注意力头个数
    "n_layers": 12,         #Transformer 层数
    "drop_rate": 0.1,       #Dropout rate
    "qkv_bias": False       #bias
}

# model = TestTransformer(GPT_CONFIG);

AttributeError: module 'torch.nn' has no attribute 'Moudle'

# 一、 推理

In [ ]:

# 
# max_new_tokens 生成文本的最大个数
# 文本上下文的大小
def generate_text(model, prompt, max_new_tokens, context_seq_size):
    for  _ in range(max_new_tokens):
        # 最大限制额 1024 
        prompt_slice = prompt[:, -context_seq_size:];
        # 创建pytorch的上下文管理器  作用是： 禁止梯度计算
        with torch.no_grad():
            # logits 嵌入向量矩阵
            logits = model(prompt);

        # 从logits中拿到最后一位
        # logits[batch_size, new_token, vocab_size];
        logits = logits[:, -1, :];

        probas = torch.softmax(logits, dim=1);

        # 得到最大概率的位置 
        new_token = torch.argmax(probas, dim=-1, keepdim=True);

        # 
        prompt = torch.cat((prompt, new_token), dim=1);
    return prompt;

# 二、 编码向量

In [ ]:
#  文本转token
def text_to_tokenids(text, tokenizer):
    encoded = tokenizer.encode(text);
    encoded_new = torch.tensor(encoded).unsqueeze(0); #  在新生成的tensor的最前面增加一个新tensor
    return encoded_new;

# token 转文本
def tokenids_to_text(tokenids, tokenizer):
    tokenids.squeeze(0); # 它会将tokenids中最前一个维度位1去了  [[1, 2, 3]]
    return tokenizer.decode(tokenids.tolist());

In [ ]:

# 设置推理模式
model.eval();


#  设置提示词
prompt = "北京的最高山是?";  # 东灵山

tokenizer = tiktoken.get_encoding("gpt2");




prpt = text_to_tokenids(prompt, tokenizer);

token_ids = generate_text(model=model, 
              prompt=prpt, 
              max_new_tokens=10, 
              context_seq_size=256, #每一句训练数据的最大长度
              );



print(tokenids_to_text(token_ids, tokenizer));